## For Binary Classification Interface

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib
import signal
from keras.models import load_model
import cv2
import numpy as np
from PIL import Image
import base64
model = load_model('binary_model.h5')
import matplotlib.pyplot as plt
def preprocessing(image):
    image=np.array(image)
    img_ben = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), 10), -3.5, 80)    
    return img_ben
def predictClass(image):
  image=np.array(image)
  y_pred_prob = model.predict(image.reshape(1,224,224,3))
  y_pred = (y_pred_prob >= 0.5).astype(int)
  y_pred=(y_pred[0])[0]
  return y_pred

def encode_image(image):
  ret, buffer = cv2.imencode('.png', image)
  return base64.b64encode(buffer.tobytes()).decode('utf-8')

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/predict', methods=['POST'])
def predict():
  try:
    uploaded_file = request.files['file']
    if uploaded_file:
      image = Image.open(uploaded_file)
      processed_image = preprocessing(image)
      ind = predictClass(image)
      classes = ['No_DR', 'DR']
      predicted_class_label = classes[ind]
      processed_image_data = encode_image(processed_image)
      return jsonify({'prediction': predicted_class_label, 'processed_image': processed_image_data})
    else:
      return jsonify({'error': 'No file uploaded'}), 400
  except Exception as e:
    return jsonify({'error': e}), 500

signal.signal(signal.SIGINT, signal.SIG_DFL)

if __name__ == '__main__':
  app.run(host='localhost', port=5000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step


INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 18:02:48] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 18:02:54] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 18:03:02] "POST /predict HTTP/1.1" 200 -


## For MultiClass Classification Interface

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib
import signal
from keras.models import load_model
import cv2
import numpy as np
from PIL import Image
import base64
model = load_model('my_model.h5')
def preprocessing(image):
    # Convert PIL image to numpy array
    image_array = np.array(image)    
    # Convert RGB to LAB color space
    lab = cv2.cvtColor(image_array, cv2.COLOR_RGB2LAB)
    # Split LAB channels
    l, a, b = cv2.split(lab)
    # Apply CLAHE to enhance L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced_l = clahe.apply(l)
    # Merge LAB channels back to RGB
    enhanced_lab = cv2.merge((enhanced_l, a, b))
    processed_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)
    return processed_image
def predictClass(image):
  processed_image = preprocessing(image)
  processed_image = np.expand_dims(processed_image, axis=0)
  prediction = model.predict(processed_image)
  return prediction

def encode_image(image):
  ret, buffer = cv2.imencode('.jpg', image)
  return base64.b64encode(buffer.tobytes()).decode('utf-8')

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/predict', methods=['POST'])
def predict():
  try:
    uploaded_file = request.files['file']
    if uploaded_file:
      image = Image.open(uploaded_file)
      processed_image = preprocessing(image)
      prediction = predictClass(processed_image)
      predicted_class_index = np.argmax(prediction)
      classes = ['Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']
      predicted_class_label = classes[predicted_class_index]
      processed_image_data = encode_image(processed_image)
      return jsonify({'prediction': predicted_class_label, 'processed_image': processed_image_data})
    else:
      return jsonify({'error': 'No file uploaded'}), 400
  except Exception as e:
    return jsonify({'error': e}), 500

signal.signal(signal.SIGINT, signal.SIG_DFL)

if __name__ == '__main__':
  app.run(host='localhost', port=5000, debug=True, use_reloader=False)